# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/MyDrive/MyDrive/PredNet_ApparentMotion

# Import

In [ ]:
!pip install torchinfo
!pip install hickle

In [ ]:
import os
import gc
import sys
import math
import numpy as np
import hickle as hkl

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader

from torchinfo import summary

from prednet import PredNet

In [ ]:
gc.collect()
torch.cuda.empty_cache()

# Setting Variables and Parameters

In [ ]:
num_epochs = 150
batch_size = 5
A_channels = (3, 48, 96, 192)
R_channels = (3, 48, 96, 192)
lr = 0.001 # if epoch < 75 0.001 else 0.0001
nt = 10 # num of time steps

In [ ]:
layer_loss_weights = Variable(torch.FloatTensor([[1.], [0.], [0.], [0.]]).cuda())
time_loss_weights = 1./(nt - 1) * torch.ones(nt, 1)
time_loss_weights[0] = 0
time_loss_weights = Variable(time_loss_weights.cuda())

In [ ]:
train_file = './kitti_data_raw/X_train.hkl'
train_sources = './kitti_data_raw/sources_train.hkl'

In [ ]:
X = hkl.load(train_file)
step = nt * batch_size
portion = math.floor(X.shape[0]/step)

# Set Model and Optimizer

In [ ]:
model = PredNet(R_channels, A_channels, output_mode='error')
if torch.cuda.is_available():
    print('Using GPU.')
    model.cuda()

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [ ]:
def lr_scheduler(optimizer, epoch):
    if epoch < num_epochs //2:
        return optimizer
    else:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.0001
        return optimizer

In [ ]:
def makeinputs(inputs):
    X = []
    ind = 0
    for b in range(batch_size):
        temp_batch = []
        for n in range(nt):
            temp_batch.append(inputs[ind].transpose(2,0,1).tolist())
            ind += 1
        X.append(temp_batch)
    return Variable(torch.Tensor(X).cuda())

# Training

In [ ]:
temporal_loss = []

for epoch in range(num_epochs):
    optimizer = lr_scheduler(optimizer, epoch)
    loss = 0
    for p in range(portion):
        Xtrim = X[p*step:(p+1)*step]
        inputs = makeinputs(Xtrim)
            
        loss = model(inputs) # batch x n_layers x nt
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (p+1)%10 == 0:
            print('step: {}/{}, loss: {}'.format(p+1,portion,loss))
            temporal_loss.append(float(loss.data.cpu().detach()))
    print('Epoch: {}/{}, loss: {}'.format(epoch+1, num_epochs, loss))
    
    with torch.no_grad():
        torch.cuda.empty_cache()
    
    if epoch%10 == 0:
        torch.save(model.state_dict(), './models/training_{:04d}.pt'.format(epoch))

torch.save(model.state_dict(), './models/training.pt')